In [1]:
import pandas as pd
import numpy as np

In [2]:
# load in sv2 merged table
# should have columns ['Chromosome', 'Type', 'Start', 'End', 'Length']
# can have more columns
svs = pd.read_csv('output/merged.PennCNV.tsv', sep='\t')

In [4]:
# downloaded from https://gnomad.broadinstitute.org/downloads#v2-structural-variants
gnomad = pd.read_csv('../sv2/gnomad_v2.1_sv.controls_only.sites.bed.gz', sep='\t')

# create gnomad column for chromosome with chr
gnomad['chrom'] = gnomad['#chrom'].apply(lambda s: 'chr{}'.format(s))

# filter for only dups and dels
gnomad = gnomad[gnomad.svtype.isin(['DEL', 'DUP'])].copy()

# filter for pass only
gnomad = gnomad[gnomad.FILTER == 'PASS'].copy()

# set gnomad frequency to type float
gnomad['AF'] = gnomad.AF.astype(float)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,22,24,36,37,46,47,61,62,72,73,82,83,96,97,106,107,116,117,130,131,140,141,150,151,164,165,174,175,184,185,198,199,208,209,218,219,232,233) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# loop over every chrom and svtype
chromosomes = list(svs.Chromosome.unique())
svtypes = ['dup', 'del']
svs['gnomad_freq_overall'] = '.'

for chrom in chromosomes:
    print(chrom)
    for svtype in svtypes:
        # get all structural variants in chromosome and svtype
        sub_svs = svs[(svs.Chromosome == chrom) & (svs.Type == svtype)].copy()
        sub_gnomad = gnomad[(gnomad['chrom'] == chrom) & (gnomad.svtype == svtype.upper())].copy()
        
        # reset index in sub_gnomad
        sub_gnomad = sub_gnomad.reset_index(drop=True)
        
        # create numpy structures for end, start, and svlength
        # (numpy is faster than pandas)
        chrom_ends = sub_gnomad['end'].to_numpy()
        chrom_starts = sub_gnomad['start'].to_numpy()
        chrom_svlengths = chrom_ends - chrom_starts

        # iterate over all structural variants in cohort
        # and compare against other structural variants gnomad
        for i, row in sub_svs.iterrows():
            start = row['Start']
            end = row['End']
            length = row['Length']

            min_end   = np.minimum(chrom_ends, end)
            max_start = np.maximum(chrom_starts, start)
            max_length = np.maximum(chrom_svlengths, length)

            overlap = (min_end - max_start) > .5 * max_length

            # in some cases the structural variant overlaps with more than one gnomad SV
            # in those cases get the sum of all SV that it overlaps with
            allele_frequency = sub_gnomad.loc[overlap]['AF'].sum()
            
            # save to table
            svs.at[i, 'gnomad_freq_control'] = allele_frequency

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX


In [6]:
svs.to_csv('output/merged.PennCNV.gnomad_control.tsv', sep='\t', index=False)